In [3]:
import pandas as pd
import numpy as np
import altair as alt
import random

In [4]:
df = pd.read_csv("https://media.githubusercontent.com/media/gprasad125/soccerDemo/main/coord_events.csv")
def to_int(coord):
    
    if not pd.isnull(coord):
        
        coord = int(coord)
    
    return coord

df['location_x'] = df['location_x'].apply(to_int)
df['location_y'] = df['location_y'].apply(to_int)

clean = df.dropna().reset_index(drop = True)
clean = clean.rename(columns = {"possession_team_name": "team",
                                "type_name": "event",
                               "location_x": "x",
                               "location_y": "y"})

clean = clean.sample(frac=1).reset_index(drop = True)

def make_color():
    
    color = "#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
    return color

event_colors = dict.fromkeys(clean['event'].unique())
for ev in event_colors:
    
    number_of_colors = 8
    color = make_color()
    if color not in event_colors.values():
        event_colors[ev] = color

In [6]:
mancity = clean[clean['team'] == "Manchester City"]
mancity.head()

,team,event,x,y
2590,Manchester City,Pressure,15.0,37.0
5053,Manchester City,Ball Receipt*,54.0,78.0
6890,Manchester City,Block,12.0,23.0
8943,Manchester City,Ball Receipt*,70.0,9.0
12140,Manchester City,Ball Receipt*,21.0,54.0


In [7]:
mancity.shape

(2725, 4)

In [11]:
#alt.data_transformers.disable_max_rows()
alt.Chart(mancity).mark_point().encode(x = "x", y = "y", color = "event")

alt.Chart(...)

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
x = clean.drop(columns = ["event"])
y = clean["event"]

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25)

In [28]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [42]:
numerical_columns = ["x", "y"]
categorical_columns = ["team"]


categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()
preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)])


In [43]:
x_train, y_train

(                        team     x     y
 3697003            Barcelona  58.0  31.0
 910487      Bristol City WFC  61.0  78.0
 2885444          Reading WFC  93.0  78.0
 1367654          Arsenal WFC  95.0  38.0
 1001922            Barcelona  72.0  46.0
 ...                      ...   ...   ...
 2829170          Chelsea FCW  52.0   5.0
 2169764            Barcelona  13.0  32.0
 1263286            Barcelona  88.0  55.0
 1797388          Utah Royals  55.0  25.0
 2167048  Manchester City WFC  68.0   1.0
 
 [2954988 rows x 3 columns],
 3697003    Ball Receipt*
 910487     Ball Receipt*
 2885444            Carry
 1367654             Pass
 1001922             Pass
                ...      
 2829170            Carry
 2169764         Pressure
 1263286    Ball Receipt*
 1797388    Ball Recovery
 2167048             Pass
 Name: event, Length: 2954988, dtype: object)

In [44]:
from sklearn.pipeline import make_pipeline

model = make_pipeline(preprocessor, LogisticRegression())
model.fit(x_train, y_train)

from sklearn import set_config
set_config(display='diagram')
model


/Users/gokul/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['team']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['x', 'y'])])),
                ('logisticregression', LogisticRegression())])

In [45]:
predictions = model.predict(x_test)

In [46]:
bool_arr = (np.array(predictions) == y_test.to_numpy())
bool_arr.mean()

0.297089531328046